In [4]:
from IPython.display import display, Image, clear_output, HTML
import ipywidgets as widgets
from jupyter_ui_poll import ui_events
import requests
from bs4 import BeautifulSoup
import json
import time
import random

random.seed(1)

# buttons helper code - start

event_info = {
    'type': '',
    'description': '',
    'time': -1
}

def wait_for_event(timeout=-1, interval=0.001, max_rate=20, allow_interupt=True):    
    start_wait = time.time()

    # set event info to be empty
    # as this is dict we can change entries
    # directly without using
    # the global keyword
    event_info['type'] = ""
    event_info['description'] = ""
    event_info['time'] = -1

    n_proc = int(max_rate*interval)+1
    
    with ui_events() as ui_poll:
        keep_looping = True
        while keep_looping==True:
            # process UI events
            ui_poll(n_proc)

            # end loop if we have waited more than the timeout period
            if (timeout != -1) and (time.time() > start_wait + timeout):
                keep_looping = False
                
            # end loop if event has occured
            if allow_interupt==True and event_info['description']!="":
                keep_looping = False
                
            # add pause before looping
            # to check events again
            time.sleep(interval)
    
    # return event description after wait ends
    # will be set to empty string '' if no event occured
    return event_info

# this function lets buttons 
# register events when clicked
def register_event(btn):
    # display button description in output area
    event_info['type'] = "click"
    event_info['description'] = btn.description
    event_info['time'] = time.time()
    return

# buttons helper code - end

# adding buttons -start

# with the helper code above we can add buttons 
# following the example below

left_right_button = ['left', 'right']

btn1 = widgets.Button(description = left_right_button[0])
btn2 = widgets.Button(description = left_right_button[1])
btn3 = widgets.Button(description="Yes")
btn4 = widgets.Button(description="No")
    
# we need to set up each button
# to call the register_event
# function when clicked
btn1.on_click(register_event) 
btn2.on_click(register_event) 
btn3.on_click(register_event) 
btn4.on_click(register_event) 

# adding buttons - end

style = 'color:green; font-size:30px;'
welcome_message = HTML(f'<h1>Welcome to the ANS test...</h1>')
welcome_message_two = HTML(f'<h1>...where we measure your approximate number sense.</h1>')
display(welcome_message)
time.sleep(3)
display(welcome_message_two)
time.sleep(3)

# data input

userid_instructions = """

Enter your anonymised ID

To generate an anonymous 4-letter unique user identifier please enter:

- two letters based on the initials (first and last name) of a childhood friend

- two letters based on the initials (first and last name) of a favourite actor / actress

e.g. if your friend was called Charlie Brown and film star was Tom Cruise

then your unique identifer would be CBTC

"""

print(userid_instructions)

userid = input("> ")

print("User entered id:", userid)

age = input("How old are you?")
gender = input("\nWhat's your gender? Enter 'f' for female, 'm' for male or 'x' for other.")

instructions = """You will see an image with different coloured dots on each side.

You will see the image for 0.75 only, after which you will need to answer

if the right side had more dots or the left side.

You will have 3 seconds to respond before you move on to the next question.

Do you understand the instructions? Type yes or no."""

print(instructions)

understand = input("> ")
if understand == 'yes':
    print("\nLet's begin!")
else:
    time.sleep(1.5)
    clear_output(wait=False)
    print(instructions)

time.sleep(1.5)
clear_output(wait=False)

empty = Image('empty.png', width = 500)
p1 = Image('14 - 12.png', width = 500)
p2 = Image('15 - 20.png', width = 500)
p3 = Image('16 - 12.png', width = 500)
p4 = Image('18 - 16.png', width = 500)
p5 = Image('18 - 20.png', width = 500)
p6 = Image('18 - 21.png', width = 500)
p7 = Image('9 - 10.png', width = 500)
p8 = Image('9 - 12.png', width = 500)

answers = {
    p1:'left',
    p2:'right',
    p3:'left',
    p4:'left',
    p5:'right',
    p6:'right',
    p7:'right',
    p8:'right'
}

all_trials = [p1, p2, p3, p4, p5, p6, p7, p8]
random.shuffle(all_trials)

display(empty)
time.sleep(1.5)
clear_output(wait=False)

score = 0

for trial in range(0, 3):
    random.shuffle(all_trials)
    display(all_trials[0])
    time.sleep(0.75)
    clear_output(wait=False)
    display(empty)

    myhtml1 = HTML("<h1>Which side had more dots?</h1>")
    display(myhtml1)
    myhtml2 = HTML("<h2>You have 3 seconds to answer...</h2>")
    display(myhtml2)
    
    panel = widgets.HBox([btn1, btn2])
    display(panel)
    
    result = wait_for_event(timeout=3)
    clear_output()

    if result['description'] != "":
        if result['description'] == answers[all_trials[0]]:
            score = score + 1
            print(f'Correct!')
            print(score)
        else:
            print(f'Incorrect!')
            print(score)
    else:
        print("Time's up!")
        print(score)

    time.sleep(1.5)
    clear_output(wait=False)

score_message = HTML(f"<h1>You've scored {score} / 40 points!</h1>")
percentage = round(score)/40
score_message_two = HTML(f"<h1>That's around {percentage}%.</h1>")
display(score_message)
time.sleep(1)
display(score_message_two)

print("Please read:")
time.sleep(1)
print("we wish to record your response data")
time.sleep(1)
print("to an anonymised public data repository. ")
time.sleep(1)
print("Your data will be used for educational teaching purposes")
time.sleep(1)
print("practising data analysis and visualisation.")
time.sleep(1)
print("")
print("Please click yes if you consent to the upload.")
time.sleep(1)

#collecting data
data_dict = {
    'userid' : userid,
    'age' : age,
    'gender' : gender,
    'score' : score,
    'percentage' : percentage
}

def send_to_google_form(data_dict, form_url):
    ''' Helper function to upload information to a corresponding google form 
        You are not expected to follow the code within this function!
    '''
    form_id = form_url[34:90]
    view_form_url = f'https://docs.google.com/forms/d/e/{form_id}/viewform'
    post_form_url = f'https://docs.google.com/forms/d/e/{form_id}/formResponse'

    page = requests.get(view_form_url)
    content = BeautifulSoup(page.content, "html.parser").find('script', type='text/javascript')
    content = content.text[27:-1]
    result = json.loads(content)[1][1]
    form_dict = {}
    
    loaded_all = True
    for item in result:
        if item[1] not in data_dict:
            print(f"Form item {item[1]} not found. Data not uploaded.")
            loaded_all = False
            return False
        form_dict[f'entry.{item[4][0][0]}'] = data_dict[item[1]]
    
    post_result = requests.post(post_form_url, data=form_dict)
    return post_result.ok

panel = widgets.HBox([btn3, btn4])
display(panel)
result = wait_for_event()
if result['description'] == "Yes":
    print("Thanks - your data will be uploaded.")
    form_url = "https://docs.google.com/forms/d/e/1FAIpQLScB1k9gHp-lYDBFHBRKTlLHg18IFS5QdXtet0mBFxX0jFq_Yg/viewform?usp=sf_link"
    send_to_google_form(data_dict, form_url)
    print("Thank you for your participation!")
else:
    print("No problem, we hope you enjoyed the test.")

# final

Please read:
we wish to record your response data
to an anonymised public data repository. 
Your data will be used for educational teaching purposes
practising data analysis and visualisation.

Please click yes if you consent to the upload.


Thanks - your data will be uploaded.
Thank you for your participation!
